In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
import gc # 垃圾回收

# 设置 Pandas 显示选项
pd.set_option('display.max_columns', None)

In [2]:
# 加载训练数据
# 注意：数据量很大 (1.15GB)，加载可能需要一点时间
try:
    df_train = pd.read_csv('train.csv')
    print("train.csv 加载成功！")
    print(f"数据形状: {df_train.shape}")
except FileNotFoundError:
    print("错误：train.csv 文件未找到。请确保它在正确的目录中。")

/tmp/ipykernel_7686/882219382.py:4: DtypeWarning: Columns (68506) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('train.csv')


train.csv 加载成功！
数据形状: (1932, 68507)


In [3]:
# 1. 转换时间列
# 这是我们进行时间序列分割的依据
df_train['t'] = pd.to_datetime(df_train['t'])

# # 2. 编码目标变量
# # 我们需要将 'H', 'L', 'None' 转换为数字
# # {'H': 0, 'L': 1, 'None': 2}
# target_map = {'H': 0, 'L': 1, 'None': 2}
# # 使用 .map() 更安全，如果出现意外值会变为 NaN
# df_train['class_label_encoded'] = df_train['class_label'].map(target_map)

# 2. 编码目标变量# 这是 *正确* 的目标映射，一个 5 分类问题# (HH=0, HL=1, LH=2, LL=3, NaN=4)
target_map = {
    'HH': 0,
    'HL': 1,
    'LH': 2,
    'LL': 3
}
# 我们将 'None' (即 NaN) 映射为 4# 先映射已知的 4 个类
df_train['class_label_encoded'] = df_train['class_label'].map(target_map)

# 然后，将原始的 NaN 值 (代表 'None') 填充为 4# .fillna(4) 只会填充 .map() 之后仍然是 NaN 的值
df_train['class_label_encoded'] = df_train['class_label_encoded'].fillna(4)

df_train['class_label_encoded'] = df_train['class_label_encoded'].astype(int)

# 检查是否有因映射失败产生的 NaN
if df_train['class_label_encoded'].isnull().any():
    print("警告：class_label 中存在未知的标签！")
    # 填充 NaN 为 'None' 对应的编码 (2)
    df_train['class_label_encoded'] = df_train['class_label_encoded'].fillna(2)

df_train['class_label_encoded'] = df_train['class_label_encoded'].astype(int)


# 3. 转换 ticker_id 为类别特征
# LightGBM 可以直接高效处理 'category' 类型的特征
df_train['ticker_id'] = df_train['ticker_id'].astype('category')

print("预处理完成。")
# print(df_train[['t', 'ticker_id', 'class_label', 'class_label_encoded']].head())
# 验证一下我们新的编码分布
print("\n--- 修正后 'class_label_encoded' 分布 ---")
print(df_train['class_label_encoded'].value_counts())

预处理完成。

--- 修正后 'class_label_encoded' 分布 ---
class_label_encoded
4    1820
0      40
1      28
3      26
2      18
Name: count, dtype: int64


In [4]:
# 定义目标变量
TARGET = 'class_label_encoded'

# 定义元数据列（这些不是特征）
METADATA_COLS = ['class_label', 'class_label_encoded', 't']

# 定义特征列
# 我们的特征是除了元数据列之外的所有列
FEATURES = [col for col in df_train.columns if col not in METADATA_COLS]

print(f"总特征数量: {len(FEATURES)}")
# 打印前 10 个特征名看看
print(f"前10个特征: {FEATURES[:10]}")
# 确认 ticker_id 在特征中
print(f"'ticker_id' 是否在特征中: {'ticker_id' in FEATURES}")

总特征数量: 68505
前10个特征: ['train_id', 'ticker_id', 'cross_threshold_from_above_100.0', 'cross_threshold_from_above_100.5', 'cross_threshold_from_above_101.0', 'cross_threshold_from_above_101.5', 'cross_threshold_from_above_102.0', 'cross_threshold_from_above_102.5', 'cross_threshold_from_above_103.0', 'cross_threshold_from_above_97.0']
'ticker_id' 是否在特征中: True


In [5]:
# 1. 确保数据严格按时间排序
# 我们按时间 't' 排序，如果时间相同，再按 'ticker_id' 排序
print("正在按 't' 和 'ticker_id' 排序数据...")
df_train = df_train.sort_values(by=['t', 'ticker_id']).reset_index(drop=True)

# 2. 定义分割点
# 我们使用 80% 的数据作为训练集，20% 作为验证集
split_percentage = 0.8
split_index = int(len(df_train) * split_percentage)

# 3. 分割数据
train_data = df_train.iloc[:split_index]
val_data = df_train.iloc[split_index:]

print(f"总样本数: {len(df_train)}")
print(f"训练集样本数: {len(train_data)} (从 {train_data['t'].min()} 到 {train_data['t'].max()})")
print(f"验证集样本数: {len(val_data)} (从 {val_data['t'].min()} 到 {val_data['t'].max()})")

# 4. 创建 X (特征) 和 y (目标)
X_train = train_data[FEATURES]
y_train = train_data[TARGET]

X_val = val_data[FEATURES]
y_val = val_data[TARGET]

# 5. 清理内存 (可选，但推荐)
del df_train, train_data, val_data
gc.collect()

print("时间序列验证集创建完毕！")

正在按 't' 和 'ticker_id' 排序数据...
总样本数: 1932
训练集样本数: 1545 (从 2023-04-03 00:00:00 到 2024-09-20 00:00:00)
验证集样本数: 387 (从 2024-09-20 00:00:00 到 2025-01-31 00:00:00)
时间序列验证集创建完毕！


In [6]:
# -----------------------------------------------------------------
# 修正 单元格 6：配置并训练模型
# -----------------------------------------------------------------

# 1. 定义 LightGBM 模型参数
lgb_params = {
    'objective': 'multiclass',  # 多分类任务
    
    # --- 这是关键修改 ---
    'num_class': 5,             # 类别数量 (HH, HL, LH, LL, None)
    # ---------------------
    
    'metric': 'multi_logloss',
    'n_estimators': 200,
    'learning_rate': 0.05,
    'n_jobs': -1,
    'seed': 42,
    
    # 类别不平衡问题 *依然* 严重 (1820 vs 112)，必须保留
    'class_weight': 'balanced'
}

# 2. 初始化模型
model = lgb.LGBMClassifier(**lgb_params)

# 3. 训练模型
print("开始训练基线模型 (5 分类)...")
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='multi_logloss',
    callbacks=[lgb.early_stopping(30)],
    categorical_feature=['ticker_id']
)

print("模型训练完成！")

开始训练基线模型...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.077369 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15504
[LightGBM] [Info] Number of data points in the train set: 1545, number of used features: 7117
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[69]	valid_0's multi_logloss: 0

In [7]:
# -----------------------------------------------------------------
# 修正 单元格 7：评估基线模型
# -----------------------------------------------------------------

# 1. 在验证集上进行预测
print("正在验证集上进行预测...")
val_preds = model.predict(X_val)

# 2. 计算 Macro F1-Score
macro_f1 = f1_score(y_val, val_preds, average='macro', zero_division=0)

print(f"========================================")
print(f"📈 基线模型 Macro F1-Score: {macro_f1:.4f}")
print(f"========================================")

# 3. 打印详细的分类报告
# 重新定义我们的映射，这次包含 'None'
target_map_full = {
    'HH': 0,
    'HL': 1,
    'LH': 2,
    'LL': 3,
    'None': 4  # 对应我们 fillna(4)
}

target_names = [key for key, val in sorted(target_map_full.items(), key=lambda item: item[1])]
all_labels = [val for key, val in sorted(target_map_full.items(), key=lambda item: item[1])]

print("\n详细分类报告 (5 分类):")

# 明确传入 'labels' 参数，确保所有 5 个类都被报告
print(classification_report(
    y_val, 
    val_preds, 
    target_names=target_names, 
    labels=all_labels,
    zero_division=0
))

正在验证集上进行预测...
📈 基线模型 Macro F1-Score: 0.1905

详细分类报告 (0:H, 1:L, 2:None):


ValueError: Number of classes, 5, does not match size of target_names, 4. Try specifying the labels parameter

In [2]:
# 安装 pandas（数据处理）
!pip install pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

# 安装 numpy（数值计算）
!pip install numpy -i https://pypi.tuna.tsinghua.edu.cn/simple

# 安装 lightgbm（梯度提升树模型，注意：Windows 可能需要先装依赖）
!pip install lightgbm -i https://pypi.tuna.tsinghua.edu.cn/simple

# 安装 scikit-learn（数据预处理、模型评估）
!pip install scikit-learn -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 41.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 135.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 109.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 17.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 20.8 MB/s eta 0:00:0000:0100:01
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 42.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 106.0 MB/s eta 0:00:00


In [11]:
# 调试过程 20251116-00:29
# -----------------------------------------------------------------# 诊断单元格：检查标签分布# -----------------------------------------------------------------# # 我们需要重新加载数据，因为在之前的流程中 df_train 可能已被删除 (gc.collect())# 如果您的 df_train 仍然存在，可以跳过加载，直接运行 .value_counts()

print("重新加载数据以进行诊断...")
try:
    df_train_diag = pd.read_csv('train.csv')
    
    print("\n--- 原始 'class_label' 分布 ---")
    # include_hidden=True 会显示 NaN 值
    print(df_train_diag['class_label'].value_counts(dropna=False))

    # 验证一下我们的映射逻辑
    target_map = {'H': 0, 'L': 1, 'None': 2}
    df_train_diag['class_label_encoded'] = df_train_diag['class_label'].map(target_map).fillna(2)
    
    print("\n--- 编码后 'class_label_encoded' 分布 ---")
    print(df_train_diag['class_label_encoded'].value_counts())
    
except FileNotFoundError:
    print("错误：train.csv 文件未找到。")

重新加载数据以进行诊断...


/tmp/ipykernel_5416/2223878534.py:6: DtypeWarning: Columns (68506) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_diag = pd.read_csv('train.csv')



--- 原始 'class_label' 分布 ---
class_label
NaN    1820
HH       40
HL       28
LL       26
LH       18
Name: count, dtype: int64

--- 编码后 'class_label_encoded' 分布 ---
class_label_encoded
2.0    1932
Name: count, dtype: int64
